In [1]:
######################################     Modules     #######################################
import sys
from time import process_time
import os

import verbose_module as verb
sys.path.append('../../.')
import pIRPgym

########################     Instance generator and Environment     #########################
# Instance Generator
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']
look_ahead = ['d','q']


# Historical data
historical_data = ['*']


# Other parameters
backorders = 'backorders'

env_config = {  'M':13,'K':15,'T':12,'F':13,'Q':2000,
                'S':6,'LA_horizon':4,
                'd_max':2000,'hist_window':60,
                'back_o_cost':10000
            }

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


# dCVRP Instances

In [2]:
########################     Instance generator and Environment     #########################
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

instances = dict()
instances['Li'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Li') if i[-3:]=='vrp']
instances['Golden'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Golden') if i[-3:]=='vrp']
instances['Li'].sort();instances['Golden'].sort()

### Environment 
# Creating environment object
routing = True
inventory = False    
perishability = False
env = pIRPgym.steroid_IRP(routing,inventory,perishability)
env.reset(inst_gen)

In [3]:
################################## Policy Evaluation ##################################
''' Parameters '''
verbose = True
show_gap = True

if verbose: verb.routing_instances.print_head(show_gap)

for ss,inst_list in instances.items():
    for instance in inst_list:
        # Upload dCVRP instance
        purchase,benchmark = inst_gen.upload_CVRP_instance(ss, instance)

        if verbose: string = verb.routing_instances.print_inst(ss,instance,benchmark[0],benchmark[1])

        nn_routes,nn_obj,nn_info,nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
        if verbose: string = verb.routing_instances.print_routing_update(string,
                                                                    nn_obj,len(nn_routes),nn_time,show_gap,benchmark)
            
        RCL_obj,RCL_veh,RCL_time = pIRPgym.Routing.evaluate_stochastic_policy(pIRPgym.Routing.RCL_Heuristic,
                                                                                           purchase,inst_gen,env,n=30,averages=True,dynamic_p=False)
        if verbose: string = verb.routing_instances.print_routing_update(string,
                                                                    RCL_obj,RCL_veh,RCL_time,show_gap,benchmark,end=True)
        # GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
        # if verbose: string = verb.routing_instances.print_routing_update(string,
        #                                                             GA_obj,len(GA_routes),GA_time,show_gap,benchmark)
        # HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                  # Hybrid Genetic Search (CVRP)
        # if verbose: string = verb.routing_instances.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes),HyGeSe_time,show_gap,benchmark,True)                                    # Column Generation algorithm
    print('\n')

*************************************  Routing Strategies on Classic Instances  *************************************
--------|	BKS 	|	   NN 	 	|	   RCL 	 	|	   HGA 	 	|	  HGS*
Inst	| #Veh 	 Obj 	| t(s)	 #Veh 	 gap 	| t(s) 	 #Veh 	gap 	| t(s) 	 #Veh 	 gap 	| t(s) 	 #Veh 	 gap 	|
----------------------------------------------------------------------------------------------------------------------


# CVRP Instances

In [3]:
########################     Instance generator and Environment     #########################
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

instances = dict()
instances = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
# instances = [i for i in os.listdir('C:/Users/jm.betancourt/Documents/Research/pIRPgym/pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
instances.sort()
instances = instances[1:] + [instances[0]]

### Environment 
# Creating environment object
routing = True
inventory = False    
perishability = False
env = pIRPgym.steroid_IRP(routing,inventory,perishability)
env.reset(inst_gen)

In [4]:
################################## Policy Evaluation ##################################
''' Parameters '''
verbose = True
show_gap = True

ranges = [i for i in range(1,11)]
performance = {'nn':{i:{'Gap':list(),'Time':list()} for i in ranges},
               'RCL':{i:{'Gap':list(),'Time':list()} for i in ranges},
               'GA':{i:{'Gap':list(),'Time':list()} for i in ranges},
               'HGS':{i:{'Gap':list(),'Time':list()} for i in ranges}}


if verbose: verb.routing_instances.print_head(show_gap)

for instance in instances:
    # Upload dCVRP instance
    purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

    if verbose: string = verb.routing_instances.print_inst('Uchoa',instance,benchmark[0],benchmark[1])

    nn_routes,nn_obj,nn_info,nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verb.routing_instances.print_routing_update(string,
                                                                nn_obj,len(nn_routes),nn_time,show_gap,benchmark)
    performance['nn'][len(purchase)//100]['Gap'].append((nn_obj-benchmark[0])/benchmark[0])
    performance['nn'][len(purchase)//100]['Time'].append(nn_time)


    RCL_obj,RCL_veh,RCL_time = pIRPgym.Routing.evaluate_stochastic_policy(pIRPgym.Routing.RCL_Heuristic,
                                                                          purchase,inst_gen,env,n=30,averages=True,dynamic_p=False)
    if verbose: string = verb.routing_instances.print_routing_update(string,
                                                                RCL_obj,RCL_veh,RCL_time,show_gap,benchmark)
    performance['RCL'][len(purchase)//100]['Gap'].append((RCL_obj-benchmark[0])/benchmark[0])
    performance['RCL'][len(purchase)//100]['Time'].append(RCL_time)

    GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,
                                                                               inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
    if verbose: string = verb.routing_instances.print_routing_update(string,
                                                                GA_obj,len(GA_routes),GA_time,show_gap,benchmark)
    performance['GA'][len(purchase)//100]['Gap'].append((GA_obj-benchmark[0])/benchmark[0])
    performance['GA'][len(purchase)//100]['Time'].append(GA_time)

    HGS_routes,HGS_distance,HGS_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                  # Hybrid Genetic Search (CVRP)
    if verbose: string = verb.routing_instances.print_routing_update(string,
                                                                               HGS_distance,len(HGS_routes),HGS_time,show_gap,benchmark,end=True)                                    # Column Generation algorithm
    performance['HGS'][len(purchase)//100]['Gap'].append((HGS_distance-benchmark[0])/benchmark[0])
    performance['HGS'][len(purchase)//100]['Time'].append(HGS_time)
    

*************************************  Routing Strategies on Classic Instances  *************************************


--------|	BKS 	|	   NN 	 	|	   RCL 	 	|	   HGA 	 	|	  HGS*
Inst	| #Veh 	 Obj 	| t(s)	 #Veh 	 gap 	| t(s) 	 #Veh 	gap 	| t(s) 	 #Veh 	 gap 	| t(s) 	 #Veh 	 gap 	|
----------------------------------------------------------------------------------------------------------------------
n101-k25| 25 	27591 	| 0.00	  26	 52.02	| 0.01	  26.7	 62.52	| 12.10	  26	 30.9	| 100.0	  26	 0.0	|
n106-k14| 14 	26362 	| 0.00	  14	 9.61	| 0.01	  14.0	 40.48	| 16.80	  14	 6.38	| 100.0	  14	 0.12	|
n110-k13| 13 	14971 	| 0.00	  13	 28.86	| 0.01	  13.0	 116.25	| 14.90	  13	 19.69	| 100.0	  13	 0.0	|
